In [235]:
import os
import shutil
from time import sleep
import time
from linecache import getline
from IPython.display import clear_output



class File:
    
    def __init__(self, file_name):
        self.delete_me = False
        self.file_name = file_name
        file_name, suffix = file_name.split(".")
        if(suffix != "out"): self.delete_me = True ; return
        self.slurm_name = file_name.split("_")[0]
            
        self.name_for_user = "Starting..."
        lines = open(self.file_name).readlines()
        for i, line in enumerate(lines):
            if(line[:5] == "name:" and lines[i+1][0] != "\t"):
                self.name_for_user = lines[i+1][:-1]
                break
        
        self.last_line = "Starting..."
        if(len(lines) != 0):
            last = -1
            while(lines[last] == "\n"): last -= 1
            self.last_line = lines[last]
            
        self.last_trim_time = time.time()
                        
    def trim_file(self):
        """Keeps only the first 1000 and last 1000 lines of the file."""
        try:
            with open(self.file_name, 'r') as f:
                lines = f.readlines()
            if len(lines) > 2000:  # Only trim if the file is too large
                with open(self.file_name, 'w') as f:
                    f.writelines(lines[:2000])  # Keep first 1000 lines
                    f.write("\n... (middle trimmed) ...\n")  # Indicate trimming
                    f.writelines(lines[-2000:])  # Keep last 1000 lines
        except Exception as e:
            print(f"Error trimming {self.file_name}: {e}")

    def check_trim(self):
        """Checks if an hour has passed and trims the file if needed."""
        if time.time() - self.last_trim_time > 3600:  # 3600 seconds = 1 hour
            self.trim_file()
            self.last_trim_time = time.time()
            
    def done(self):
        if(self.last_line.endswith("Done!\n")): return(True)
        return(False)
        
    def __lt__(self, other): return(self.slurm_name < other.slurm_name)
            
    

first = True
while(True):
    file_names = [f for f in os.listdir() if f[:5]=="slurm"]
    
    files = []
    for name in file_names:
        file = File(name)
        if(file.delete_me): pass 
        else:               files.append(file)
    files.sort()

    ongoing = []              ; finished = []
    
    for file in files:
        done = file.done() 
        #file.check_trim()
        if(done): finished.append(file)
        else:     ongoing.append(file)
    
    if(not first):
        print("\n\n")
        for i in range(10):
            print(i+1, end = "... ")
            sleep(1)
    first = False
    
    clear_output(wait=True)
    
    uniform_length = 0
    for file in ongoing + finished:
        if(file.last_line != "\n"):
            label = "{} ({}):".format(file.name_for_user, file.file_name)
            uniform_length = max([uniform_length, len(label)])
                
    print("Ongoing:")
    if(len(ongoing) == 0): print("None.")
    for file in ongoing: 
        label = "{} ({}):".format(file.name_for_user, file.file_name)
        separation = " " * (uniform_length - len(label))
        last_line = file.last_line if file.last_line[-1] != "\n" else file.last_line[:-1]
        print("{}{}\t{}".format(label, separation, last_line))

    print("\n\n\nFinished:")
    if(len(finished) == 0): print("None.")
    else:
        for i, file in enumerate(finished):
            label = "{} ({}):".format(file.name_for_user, file.file_name)
            separation = " " * (uniform_length - len(label))
            last_line = file.last_line if file.last_line[-1] != "\n" else file.last_line[:-1]
            print("{}{}\t{}".format(label, separation, last_line))
            
    finished_names = [file.name_for_user for file in finished]
    ongoing_names = [file.name_for_user for file in ongoing]

Ongoing:
plotting (slurm-59958438.out):              	  if(key != "xs"): log_quantile_dict[key] = np.log(quantile_dict[key])



Finished:
e (slurm-59958428.out):                     	Duration: 1 day, 8:05:16.005998. Done!
e (slurm-59958429.out):                     	Duration: 1 day, 13:13:17.758840. Done!
e (slurm-59958430.out):                     	Duration: 1 day, 10:39:09.539902. Done!
ec (slurm-59958431.out):                    	Duration: 1 day, 7:18:42.153380. Done!
ec (slurm-59958432.out):                    	Duration: 1 day, 9:49:21.584621. Done!
ec (slurm-59958433.out):                    	Duration: 1 day, 9:17:50.254351. Done!
ef (slurm-59958434.out):                    	Duration: 1 day, 3:40:24.542019. Done!
ef (slurm-59958435.out):                    	Duration: 1 day, 14:14:27.247008. Done!
ef (slurm-59958436.out):                    	Duration: 1 day, 7:26:28.680353. Done!
finishing_dictionaries (slurm-59958437.out):	Duration: 0:01:36.196126. Done!
plotting_components (slurm

In [234]:
#arg_title_func = "finish_dicts"
arg_title_func = "plotting"
#arg_title_func = "plotting_components_w_nn"

if len(finished_names) > 0:
    print("\n\nFINISHED Generated arg_title:")
    print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{'+'.join(finished_names)}___ --arg_name {arg_title_func}")
    print()
    for finished_name in finished_names:
        print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{finished_name}___ --arg_name {arg_title_func}")
    print("\n\nFINISHED Generated arg_list:")
    print(f"bash communication/bash/maze.sh 'arg_list=({' '.join(finished_names)}' 'comp=deigo' 'agents=0'")
    print()
    for finished_name in finished_names:
        print(f"bash communication/bash/maze.sh 'arg_list=({finished_name}' 'comp=deigo' 'agents=0'")
        
        

print("\n\nONGOING Generated arg_title:")
print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{'+'.join(ongoing_names)}___ --arg_name {arg_title_func} --temp True")
print()
for ongoing_name in ongoing_names:
    print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{ongoing_name}___ --arg_name {arg_title_func} --temp True")
print("\n\nONGOING Generated arg_list:")
print(f"bash communication/bash/maze.sh 'arg_list=({' '.join(ongoing_names)}' 'comp=deigo' 'agents=0'")
print()
for ongoing_name in ongoing_names:
    print(f"bash communication/bash/maze.sh 'arg_list=({ongoing_name}' 'comp=deigo' 'agents=0'")
    
    
    
print("\n\nALL Generated arg_title:")
print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{'+'.join(finished_names) + '+' + '+'.join(ongoing_names)}___ --arg_name {arg_title_func} --temp True")
print()
for name in finished_names + ongoing_names:
    print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{name}___ --arg_name {arg_title_func} --temp True")
print("\n\nALL Generated arg_list:")
print(f"bash communication/bash/maze.sh 'arg_list=({' '.join(finished_names) + ' ' + ' '.join(ongoing_names)}' 'comp=deigo' 'agents=0'")
print()
for ongoing_name in finished_names + ongoing_names:
    print(f"bash communication/bash/maze.sh 'arg_list=({ongoing_name}' 'comp=deigo' 'agents=0'")




FINISHED Generated arg_title:
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___e+e+ec+ec+ec+ef+ef___ --arg_name plotting

singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___e___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___e___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ec___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ec___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ec___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ef___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ef___ --arg_name plotting


FINISHED Generated arg_list:
bash communication